# Investigating Fandango Movie Ratings

Back in October 2015, a data journalist from [FiveThirtyEight](https://fivethirtyeight.com) named Walt Hickey published [this analysis](https://fivethirtyeight.com/features/fandango-movies-ratings/) that suggests movie ratings on [Fandango](https://www.fandango.com) were dishonest and rated higher than they should be. He was able to find that the actual ratings in Fandango's HTML were nearly always displayed to the user rounded up to the closest half-star and sometimes rounded up a full star.

After Hickey's analysis was published, Fandango claimed that the biased rounding was caused by a bug in their system, but we can no longer tell for sure since the actual rating value no longer shows up in the page's HTML.

In this project, we'll analyze some more recent movie ratings to see if we can determine whether or not Fandango has changed their rating system. We can do this by analysing Walt Hickey's original data set on GitHub [here](https://github.com/fivethirtyeight/data/tree/master/fandango), and comparing it to movie ratings data from movies released in 2016 and 2017 [here](https://github.com/mircealex/Movie_ratings_2016_17).

### Taking a First Look at the Data


We'll begin by reading in the data sets and looking at their structure.

In [6]:
import pandas as pd

pd.options.display.max_columns = 100 # To be able to view all columns within this notebook

before = pd.read_csv('fandango_score_comparison.csv')
after = pd.read_csv('movie_ratings_16_17.csv')

before.head()

,FILM,RottenTomatoes,RottenTomatoes_User,Metacritic,Metacritic_User,IMDB,Fandango_Stars,Fandango_Ratingvalue,RT_norm,RT_user_norm,Metacritic_norm,Metacritic_user_nom,IMDB_norm,RT_norm_round,RT_user_norm_round,Metacritic_norm_round,Metacritic_user_norm_round,IMDB_norm_round,Metacritic_user_vote_count,IMDB_user_vote_count,Fandango_votes,Fandango_Difference
0,Avengers: Age of Ultron (2015),74,86,66,7.1,7.8,5.0,4.5,3.70,4.3,3.30,3.55,3.90,3.5,4.5,3.5,3.5,4.0,1330,271107,14846,0.5
1,Cinderella (2015),85,80,67,7.5,7.1,5.0,4.5,4.25,4.0,3.35,3.75,3.55,4.5,4.0,3.5,4.0,3.5,249,65709,12640,0.5
2,Ant-Man (2015),80,90,64,8.1,7.8,5.0,4.5,4.00,4.5,3.20,4.05,3.90,4.0,4.5,3.0,4.0,4.0,627,103660,12055,0.5
3,Do You Believe? (2015),18,84,22,4.7,5.4,5.0,4.5,0.90,4.2,1.10,2.35,2.70,1.0,4.0,1.0,2.5,2.5,31,3136,1793,0.5
4,Hot Tub Time Machine 2 (2015),14,28,29,3.4,5.1,3.5,3.0,0.70,1.4,1.45,1.70,2.55,0.5,1.5,1.5,1.5,2.5,88,19560,1021,0.5


In [5]:
after.head()

,movie,year,metascore,imdb,tmeter,audience,fandango,n_metascore,n_imdb,n_tmeter,n_audience,nr_metascore,nr_imdb,nr_tmeter,nr_audience
0,10 Cloverfield Lane,2016,76,7.2,90,79,3.5,3.80,3.60,4.50,3.95,4.0,3.5,4.5,4.0
1,13 Hours,2016,48,7.3,50,83,4.5,2.40,3.65,2.50,4.15,2.5,3.5,2.5,4.0
2,A Cure for Wellness,2016,47,6.6,40,47,3.0,2.35,3.30,2.00,2.35,2.5,3.5,2.0,2.5
3,A Dog's Purpose,2017,43,5.2,33,76,4.5,2.15,2.60,1.65,3.80,2.0,2.5,1.5,4.0
4,A Hologram for the King,2016,58,6.1,70,57,3.0,2.90,3.05,3.50,2.85,3.0,3.0,3.5,3.0
